Recherche et extraction des entités nommées avec Transformers.
Modèle de language utilisé: (Camem)BERT.

In [ ]:
# @title Installation des bibliothèques, connexion au Google Drive

"""Marina Giardinetti
marina.giardinetti@gmail.com
Google Colab
2024"""

from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re
import os
import glob
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner")

##### Process text sample (from wikipedia)

nlp = pipeline('ner', model=model, tokenizer=tokenizer, aggregation_strategy="simple")


In [ ]:
# @title Chargement du corpus
liste_ner =[]

working_dir= '/content/drive/My Drive/NER_BNF/' #@param {type:"string"}
corpus = '/content/drive/My Drive/NER_BNF/corpus2/' #@param {type:"string"}

# Afficher le nombre de textes

textes = os.listdir(corpus)
print('Nombre de textes chargés dans le corpus: ', len(textes))


In [ ]:
# @title Recherche et extraction des entités nommées
for texte in glob.glob(''.join([corpus, '*.txt'])):
  corpus = open(texte)
  corpus = corpus.read()

  res = [x for x in re.split("[//.|//!|//?]", corpus) if x!=""]
  for sentence in res:
    ner = nlp(sentence)
    liste_ner.append(ner)
liste_ner2 =[]
for el in liste_ner:
  for el2 in el:
    liste_ner2.append(el2)


df = pd.DataFrame.from_records(liste_ner2)
#@markdown Indiquer le taux de sureté minimal requis pour chaque entité detectée
taux_de_precision = 0.85 #@param {type:"string"}

df_pers = df.loc[(df['entity_group'] == 'PER') & (df['score'] >= taux_de_precision)]
df_pers.to_excel(working_dir, 'bert_pers.xlsx')

df_loc = df.loc[(df['entity_group'] == 'LOC') & (df['score'] >= taux_de_precision)]
df_loc.to_excel(working_dir, 'bert_lieux.xlsx')